In [ ]:
import time
import pandas as pd
import requests

# (가상) API 키 — 실제 사용 시 본인 키로 교체
API_KEY = "696f359837c09357926367djs8ce520"

INPUT_CSV = "restaurants.csv"
OUTPUT_CSV = "restaurants_with_latlon.csv"

def geocode_mapsco(address: str):
    """geocode.maps.co로 주소 -> (lat, lon). 실패 시 (None, None)"""
    url = "https://geocode.maps.co/search"
    params = {
        "q": address,
        "api_key": API_KEY,   # 키 없는 무료도 동작할 수 있지만, 키 사용을 기본으로 둠
    }

    r = requests.get(url, params=params, timeout=15)
    r.raise_for_status()

    data = r.json()
    if not data:
        return None, None

    # 보통 첫 결과가 가장 근접
    lat = float(data[0]["lat"])
    lon = float(data[0]["lon"])
    return lat, lon

def pick_column(df, candidates):
    """df.columns 안에서 후보 중 존재하는 첫 컬럼명 반환"""
    for c in candidates:
        if c in df.columns:
            return c
    return None

def main():
    df = pd.read_csv(INPUT_CSV)
    print("CSV 컬럼명:", list(df.columns))

    name_col = pick_column(df, ["name", "Name", "식당명", "상호", "상호명"])
    addr_col = pick_column(df, ["address", "adress", "Address", "주소"])

    if addr_col is None:
        raise ValueError("주소 컬럼을 못 찾았어요. CSV에 address(또는 adress/주소) 컬럼이 있어야 합니다.")
    if name_col is None:
        print("식당명 컬럼(name)을 못 찾았어요. 지도 팝업에 이름이 비어도 진행은 됩니다.")
        name_col = None

    # lat/lon 컬럼 생성(있으면 덮어쓰기)
    df["lat"] = None
    df["lon"] = None

    success = 0
    fail = 0

    for i, row in df.iterrows():
        addr = str(row[addr_col]).strip()

        # 빈 주소는 스킵
        if not addr or addr.lower() == "nan":
            fail += 1
            continue

        try:
            lat, lon = geocode_mapsco(addr)
            df.at[i, "lat"] = lat
            df.at[i, "lon"] = lon
            if lat is None:
                fail += 1
            else:
                success += 1
        except Exception as e:
            # 네트워크/응답 오류 등
            df.at[i, "lat"] = None
            df.at[i, "lon"] = None
            fail += 1

        # 무료/공유 API는 텀을 두는 게 안전 (차단 방지)
        time.sleep(0.8)

    df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
    print(f"완료: {OUTPUT_CSV} 저장")
    print(f"좌표 성공: {success}개 / 실패: {fail}개")

    # 실패한 행만 따로 보고 싶으면 이 파일도 생성(선택)
    fail_df = df[df["lat"].isna() | df["lon"].isna()].copy()
    if len(fail_df) > 0:
        fail_df.to_csv("geocode_failed.csv", index=False, encoding="utf-8-sig")
        print(f"참고: 실패 목록 geocode_failed.csv 저장 ({len(fail_df)}개)")

if __name__ == "__main__":
    main()
